In [284]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Load data

In [285]:
df_label = pd.read_csv('../data/label.csv')
df_sub = pd.read_csv('../data/sample_submission.csv')
df_sensor_log = pd.read_csv('../data/sensor_log.csv')

# df_new_data = pd.read_csv('../analysis/new_data/new_data.csv')
# df_new_label = pd.read_csv('../analysis/new_data/df_new_data_label.csv')

In [286]:
# df_sensor_log = pd.concat([df_sensor_log, df_new_data], axis=0)
# df_label = pd.concat([df_label, df_new_label], axis=0)
# df_sensor_log.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-01 00:00:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
1,2018-04-01 00:01:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
2,2018-04-01 00:02:00,0,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013
3,2018-04-01 00:03:00,0,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247
4,2018-04-01 00:04:00,0,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247


### Process noise

In [288]:
import numpy as np
def denoise(df):
    for col in df.columns:
        if col not in ['timestamp','block_id']:
            df[col] = np.floor(df[col]*100) / 100
    return df

# df_sensor_log = denoise(df_sensor_log)

In [289]:
## feature for trainning
num_features = []

### Function process data

In [290]:
num_feat = list(df_sensor_log.columns)[2:]
print(num_feat)

['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09']


In [291]:
def get_last(df, num_feat):
    df_last = df.groupby('block_id')[num_feat].last().add_prefix('last_')

    return df_last

def get_diff(df, num_feat):
    diff = df_sensor_log.groupby('block_id')[num_feat].diff().add_prefix('tmp_')
    diff['block_id'] = df['block_id'].values
    diff.dropna(inplace=True)

    num_new_feat = list(diff.columns)

    df_diff = diff.groupby('block_id')[num_new_feat].mean().add_prefix('diff_')

    return df_diff

def get_mean(df, num_feat):
    df_mean = df.groupby('block_id')[num_feat].mean().add_prefix('mean_')

    return df_mean

def get_sum(df, num_feat):
    df_sum = df.groupby('block_id')[num_feat].sum().add_prefix('sum_')

    return df_sum

def get_std(df, num_feat):
    df_std = df.groupby('block_id')[num_feat].std().add_prefix('std_')

    return df_std

def get_max(df, num_feat):
    df_max = df.groupby('block_id')[num_feat].max().add_prefix('max_')

    return df_max

def get_min(df, num_feat):
    df_min = df.groupby('block_id')[num_feat].min().add_prefix('min_')

    return df_min
    

In [292]:
df_mean = get_mean(df_sensor_log, num_feat)
df_mean.head(2)

,mean_sensor_00,mean_sensor_01,mean_sensor_02,mean_sensor_03,mean_sensor_04,mean_sensor_05,mean_sensor_06,mean_sensor_07,mean_sensor_08,mean_sensor_09
block_id,,,,,,,,,,
0,2.454966,47.131073,53.185760,46.380206,634.75692,75.89820,13.358651,16.155959,15.698062,15.084637
1,2.455064,48.315972,53.203124,46.362845,636.80556,77.24151,13.357204,16.168978,15.709636,15.086083


In [293]:
df_std = get_std(df_sensor_log, num_feat)
df_std.head(2)

,std_sensor_00,std_sensor_01,std_sensor_02,std_sensor_03,std_sensor_04,std_sensor_05,std_sensor_06,std_sensor_07,std_sensor_08,std_sensor_09
block_id,,,,,,,,,,
0,0.008337,0.085470,0.022412,0.036601,4.060650,1.356131,0.061103,0.064360,0.207101,0.041909
1,0.007750,0.357206,0.172160,0.044827,5.397919,1.693346,0.036451,0.026592,0.146793,0.050836


In [294]:
df_max = get_max(df_sensor_log, num_feat)
df_max.head(2)

,max_sensor_00,max_sensor_01,max_sensor_02,max_sensor_03,max_sensor_04,max_sensor_05,max_sensor_06,max_sensor_07,max_sensor_08,max_sensor_09
block_id,,,,,,,,,,
0,2.465394,47.35243,53.21180,46.39757,641.7823,78.18568,13.43316,16.24711,16.19647,15.16204
1,2.464410,48.61111,53.68924,46.39757,644.3287,80.65949,13.41146,16.21094,15.89265,15.16927


In [295]:
df_min = get_min(df_sensor_log, num_feat)
df_min.head(2)

,min_sensor_00,min_sensor_01,min_sensor_02,min_sensor_03,min_sensor_04,min_sensor_05,min_sensor_06,min_sensor_07,min_sensor_08,min_sensor_09
block_id,,,,,,,,,,
0,2.444734,47.04861,53.1684,46.31076,628.1250,73.54598,13.25231,16.03733,15.47309,15.01013
1,2.444734,47.48264,53.1250,46.31076,630.0926,74.58654,13.30295,16.13136,15.45139,15.01013


In [296]:
df_sum = get_sum(df_sensor_log, num_feat)
df_sum.head(2)

,sum_sensor_00,sum_sensor_01,sum_sensor_02,sum_sensor_03,sum_sensor_04,sum_sensor_05,sum_sensor_06,sum_sensor_07,sum_sensor_08,sum_sensor_09
block_id,,,,,,,,,,
0,24.549655,471.31073,531.857599,463.802064,6347.5692,758.9820,133.58651,161.55959,156.98062,150.84637
1,24.550638,483.15972,532.031240,463.628450,6368.0556,772.4151,133.57204,161.68978,157.09636,150.86083


In [297]:
df_all = pd.concat([df_mean, df_std, df_max, df_min, df_sum], axis=1).reset_index()
df_all

,block_id,mean_sensor_00,mean_sensor_01,mean_sensor_02,mean_sensor_03,mean_sensor_04,mean_sensor_05,mean_sensor_06,mean_sensor_07,mean_sensor_08,...,sum_sensor_00,sum_sensor_01,sum_sensor_02,sum_sensor_03,sum_sensor_04,sum_sensor_05,sum_sensor_06,sum_sensor_07,sum_sensor_08,sum_sensor_09
0,0,2.454966,47.131073,53.185760,46.380206,634.756920,75.898200,13.358651,16.155959,15.698062,...,24.549655,471.310730,531.857599,463.802064,6347.569200,758.982000,133.586510,161.559590,156.980620,150.846370
1,1,2.455064,48.315972,53.203124,46.362845,636.805560,77.241510,13.357204,16.168978,15.709636,...,24.550638,483.159720,532.031240,463.628450,6368.055600,772.415100,133.572040,161.689780,157.096360,150.860830
2,2,2.452899,48.897568,53.124998,46.271700,634.826370,76.463261,13.360819,16.166810,15.760272,...,24.528994,488.975680,531.249980,462.717004,6348.263700,764.632610,133.608190,161.668100,157.602720,150.766800
3,3,2.456441,48.559026,53.107636,46.210936,633.067110,73.055601,13.340565,16.147277,15.745082,...,24.564410,485.590260,531.076365,462.109359,6330.671100,730.556010,133.405650,161.472770,157.450820,150.817430
4,4,2.453391,49.118921,53.107636,46.098090,634.201410,74.300801,13.318139,16.142214,15.585937,...,24.533912,491.189210,531.076361,460.980896,6342.014100,743.008010,133.181390,161.422140,155.859370,150.947620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23278,23278,2.528952,47.453413,52.225584,43.682900,625.237300,75.884834,13.266397,16.194414,15.620579,...,25.289522,474.534134,522.255843,436.829000,6252.373002,758.848337,132.663975,161.944142,156.205791,151.652579
23279,23279,2.490957,48.396053,51.930535,44.651632,651.297151,76.233073,13.856167,16.127691,15.328035,...,24.909567,483.960529,519.305353,446.516324,6512.971505,762.330733,138.561669,161.276911,153.280350,152.788549
23280,23280,2.462449,48.443648,51.729567,44.805118,608.882349,75.509169,13.874082,16.154396,15.546874,...,24.624487,484.436485,517.295668,448.051184,6088.823495,755.091691,138.740817,161.543961,155.468743,151.606380
23281,23281,2.462992,47.768591,52.900923,44.394757,644.717214,71.548002,13.365121,16.009112,15.362656,...,24.629921,477.685905,529.009228,443.947568,6447.172139,715.480022,133.651211,160.091123,153.626564,152.002735


### Split data train, test

In [298]:
## Split train test
blockid_train = df_label['block_id'].values
_train = df_all['block_id'].isin(blockid_train)
df_train = df_all[_train].reset_index(drop=True)
df_test = df_all[~_train].reset_index(drop=True)

df_train.head(2)

,block_id,mean_sensor_00,mean_sensor_01,mean_sensor_02,mean_sensor_03,mean_sensor_04,mean_sensor_05,mean_sensor_06,mean_sensor_07,mean_sensor_08,...,sum_sensor_00,sum_sensor_01,sum_sensor_02,sum_sensor_03,sum_sensor_04,sum_sensor_05,sum_sensor_06,sum_sensor_07,sum_sensor_08,sum_sensor_09
0,144,2.454769,45.694442,52.32205,44.826386,634.57176,79.325703,13.093894,16.158127,15.633679,...,24.547687,456.94442,523.2205,448.263865,6345.7176,793.25703,130.93894,161.58127,156.33679,151.04888
1,145,2.454572,45.815969,52.33073,44.900174,632.03701,81.686038,13.027343,16.159576,15.632958,...,24.545719,458.15969,523.3073,449.001735,6320.3701,816.86038,130.27343,161.59576,156.32958,150.78848


In [299]:
block_id = df_label['block_id'].values
anomalous = df_label['anomalous'].values

x_train, x_test, y_train, y_test = train_test_split(block_id, anomalous, test_size=0.2, random_state=42)

len(x_train), len(x_test)

(7824, 1957)

In [300]:
data_train = df_train[df_train['block_id'].isin(x_train)]
data_val = df_train[df_train['block_id'].isin(x_test)]

data_train_label = df_label[df_label['block_id'].isin(x_train)]
data_val_label = df_label[df_label['block_id'].isin(x_test)]

len(data_train) + len(data_val), len(data_train_label) + len(data_val_label)

(9781, 9781)

### Trainning

In [301]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVC

In [302]:
num_features = df_all.columns.to_list()[1:]

In [303]:
train = data_train[num_features].values
val = data_val[num_features].values

y_train = data_train_label['anomalous'].values
y_val = data_val_label['anomalous'].values

In [ ]:
# clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
clf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=15, random_state=42, verbose=10, n_estimators=100))
clf.fit(train, y_train)

In [ ]:
result = clf.predict_proba(val)

In [306]:
y_pred = result[:, 1]
y_pred

array([0.00831155, 0.06299468, 0.00800606, ..., 1.        , 1.        ,
       1.        ])

In [307]:
val_metric = roc_auc_score(y_val, y_pred)

In [308]:
val_metric

0.9951784293730989

In [309]:
test = df_test[num_features].values

In [310]:
result = clf.predict_proba(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [311]:
anomalous = result[:, 1]

In [ ]:
df_sub

In [313]:
df_sub['anomaly_score'] = anomalous

In [ ]:
df_sub

In [315]:
df_sub.to_csv('submission_02.csv', index=False)

### Kfold

In [329]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
kf.get_n_splits(block_id)
clf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=15, random_state=42, verbose=10, n_estimators=100))

num_features = df_all.columns.to_list()[1:]
result_anomalous = []

block_id = df_label['block_id'].values
anomalous = df_label['anomalous'].values

for i, (train_index, test_index) in enumerate(kf.split(block_id)):
    print("Fflod: ", i)

    x_train, x_test, y_train, y_test = block_id[train_index], block_id[test_index], anomalous[train_index], anomalous[test_index]
    
    data_train = df_train[df_train['block_id'].isin(x_train)]
    data_val = df_train[df_train['block_id'].isin(x_test)]

    data_train_label = df_label[df_label['block_id'].isin(x_train)]
    data_val_label = df_label[df_label['block_id'].isin(x_test)]

    train = data_train[num_features].values
    val = data_val[num_features].values

    y_train = data_train_label['anomalous'].values
    y_val = data_val_label['anomalous'].values
    
    clf.fit(train, y_train)

    result_val = clf.predict_proba(val)
    
    y_pred = result_val[:, 1]
    try:
        val_metric = roc_auc_score(y_val, y_pred)
    except:
        continue

    if val_metric > 90:
        test = df_test[num_features].values
        result_test = clf.predict_proba(test)
        anomalous_test = result_test[:, 1]
        result_anomalous.append(anomalous_test)

    print("Val metric: ", val_metric)


Fflod:  0
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s


building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s


building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55

...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 

building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s


building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 o

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [330]:
len(result_anomalous)

6

In [331]:
result_anomalous[0].shape

(13502,)

In [332]:
anomalous_score = np.vstack(result_anomalous)

In [333]:
anomalous_score_mean = np.mean(anomalous_score, axis=0)

In [334]:
anomalous_score_mean.shape

(13502,)

In [336]:
df_sub['anomaly_score'] = anomalous_score_mean
df_sub.to_csv('submission_03.csv', index=False)